In [36]:
import keras
import numpy as np
from keras.datasets import fashion_mnist
from tqdm.auto import tqdm

In [8]:
!pip install tensorflow

# Question 1

In [49]:
def load_fashion_mnist(return_images=False):

  (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

  train_shuffler = np.random.shuffle(np.arange(50000))
  x_train, y_train = x_train[train_shuffler][0], y_train[train_shuffler][0]

  test_shuffler = np.random.shuffle(np.arange(10000))
  x_test, y_test = x_test[test_shuffler][0], y_test[test_shuffler][0]

  x_train = np.array(x_train/255).astype('float32')
  x_test = np.array(x_test/255).astype('float32')

  x_train, x_val = x_train[:50000], x_train[50000:]
  y_train, y_val = y_train[:50000], y_train[50000:]


  if (return_images==False):
    return {
        'train': {
            'X': x_train.reshape([50000, 784]),
            'Y': y_train.reshape([50000])
        },
        'val': {
            'X': x_val.reshape([10000, 784]),
            'Y': y_val.reshape([10000])
        },
        'test': {
            'X': x_test.reshape([10000, 784]),
            'Y': y_test.reshape([10000])
        }
  }

  else :
    return {
      'train': {
          	'X': x_train,
          	'Y': y_train
      },
      'val': {
            'X': x_val,
            'Y': y_val
      },
      'test': {
            'X': x_test,
            'Y': y_test
      }
    }


data = load_fashion_mnist()

# Question 2

In [72]:
class neural_network:

  # constructor function - initializes weights
  def __init__(self, dict_layers, initializer):

    self.weights_list = []
    self.biases_list = []
    self.dict_layers= dict_layers

    self.weights_list, self.biases_list = wandb_initializer(dict_layers, self.weights_list, self.biases_list, initializer)

  # function to compute forward propogation
  def forward_prop(self, W, b, X, Y, activation_func):

    A = []
    H = []
    
    H_pre = X
    
    L = self.dict_layers['num_hidden_layers']

    for i in range(L) :
      A.append(W[i] @ H_pre + b[i])
      H_pre = getattr(activation, activation_func)(A[i])
      H.append(H_pre)
    
    A.append(W[L] @ H_pre + b[L])
    
    Y_hat = activation.softmax(A[L])
    
    return {
        'A' : A,
        'H' : H,
        'Y_hat' : Y_hat
    }

  def self_forward_prop(self, X, Y, activation_func) :

    temp = self.forward_prop(self.weights_list,self.biases_list, X, Y, activation_func)
    return temp

  def back_prop(self, W, b, A, H, Y_hat, X, Y,activation_func):

    batch_size = len(Y)
    
    del_w = []
    del_b = []

    L = self.dict_layers['num_hidden_layers']
    
    E = np.zeros(Y_hat.shape)
    
    # E[np.arange(Y.size), Y] = 1
    for j in range(len(Y)):
        E[int(Y[j])][j] = 1
    
    # what shape do you need y_hat and e to be in? Column or row vector?
    grad_A = -(E - Y_hat)
    #print('grad_a', grad_a.shape)

    for i in range(L,-1,-1) :

      temp1 = grad_A.reshape(-1,batch_size)
      if i==0 :
        temp2 = X.reshape((batch_size ,-1))
      else :
        temp2 = H[i-1].reshape((batch_size ,-1))

      del_w.append(temp1 @ temp2)
      del_b.append(grad_A)

      if(i!=0) :
        grad_H = W[i].T @ grad_A      
        act = activation()
        grad_A = grad_H * getattr(act,activation_func+'_der')(H[i-1])

    return {
        'dw' : del_w,
        'db' : del_b
    }

  def self_back_prop(self, A, H, Y_hat, X, Y,activation_func) :
    temp = self.back_prop(self.weights_list,self.biases_list, A, H, Y_hat, X, Y, activation_func)
    return temp

  def grad_wandb(self, W, b, X, Y,activation_func):

    X = X.T.reshape((784,-1))
    Y = Y.reshape((-1, 1))
    
    temp = self.forward_prop(W, b, X, Y, activation_func)
    temp2 = self.back_prop(W, b, temp['A'], temp['H'], temp['Y_hat'], X, Y, activation_func)

    return {
        'dw' : temp2['dw'],
        'db' : temp2['db']
    }

  def self_grad_wandb(self, X, Y, activation_func) :
    temp = self.grad_wandb(self.weights_list, self.biases_list, X, Y,activation_func)
    return temp

  def predict(self, X, activation_func):
    temp = self.forward_prop(self.weights_list,self.biases_list, X, 0, activation_func)
    return {
      'y' : np.argmax(temp['Y_hat']),
      'y_hat' : temp['Y_hat']
    }

  def update_vals(self,dw,db) :
    L = len(self.weights_list)
    for i in range(L) :
      self.weights_list[i] =self.weights_list[i] - dw[L-i-1].reshape(self.weights_list[i].shape)

    for i in range(len(self.biases_list)) :
      self.biases_list[i] =self.biases_list[i] - db[L-i-1].reshape(self.biases_list[i].shape)
##################################################################################
class activation:
  
  @staticmethod
  def sigmoid(z):
    z = np.array(z,dtype=np.longdouble)
    return 1 / (1 + np.exp(-z))
  
  @staticmethod
  def relu(z):
    return (z>0) * z

  @staticmethod
  def tanh(z):
    return np.tanh(z)

  @staticmethod
  def sigmoid_der(z) :
    return z * (1-z)
  
  @staticmethod
  def relu_der(z) :
    return (z>0)

  @staticmethod
  def tanh_der(z):
    return 1 - z*z

  @staticmethod
  def softmax(x):
    x = np.array(x,dtype=np.longdouble)
    e_x = np.exp(x - np.max(x)+200)
    return e_x / e_x.sum()

def set_nn_shape(verbose=True, num_hidden_layers=-1, hidden_layer_size=-1):

  input_layer_size = 784
  hidden_layer_size = hidden_layer_size
  num_hidden_layers = num_hidden_layers
  output_layer_size = 10

  if (verbose):
    print("\nNumber Of Hidden Layers:")
    num_hidden_layers = int(input())

    print("\nSize Of Each Hidden Layer:")
    hidden_layer_size = int(input())

    print(f"\nThe Neural Network Has {num_hidden_layers+2} Layers In Total!")
  
  return {"input_layer_size": input_layer_size, "hidden_layer_size": hidden_layer_size, "output_layer_size": output_layer_size, "num_hidden_layers": num_hidden_layers}


In [70]:
import tensorflow as tf

def wandb_initializer(nn_shape, weights_list, biases_list, type='random', mu = 0, sigma = 1):
  
  # random initialization
  if (type=='random'):
    initializer = tf.keras.initializers.TruncatedNormal(mean=mu, stddev=sigma)
  # xavier initialization
  elif (type=='xavier'):
    initializer = tf.keras.initializers.GlorotNormal()

  weights_list.append(initializer(shape=(nn_shape['hidden_layer_size'], nn_shape['input_layer_size'])).numpy())
  biases_list.append(initializer(shape=(nn_shape['hidden_layer_size'], 1)).numpy())
  for i in range(nn_shape['num_hidden_layers'] - 1):
    weights_list.append(initializer(shape=(nn_shape['hidden_layer_size'], nn_shape['hidden_layer_size'])).numpy())
    biases_list.append(initializer(shape=(nn_shape['hidden_layer_size'], 1)).numpy())

  weights_list.append(initializer(shape=(nn_shape['output_layer_size'], nn_shape['hidden_layer_size'])).numpy())
  biases_list.append(initializer(shape=(nn_shape['output_layer_size'], 1)).numpy())

  return weights_list, biases_list

In [84]:
class optimizer:

  @staticmethod
  def sgd_old(network, data, config):

    # num_hidden_layers, hidden_layers_size = config['num_hidden_layers'], config['hidden_layer_size']
    num_epochs, batch_size = config['num_epochs'], config['batch_size']
    eta, lambda_ = config['lr'], config['weight_decay']
    initializer, activation_func = config['weights_initializer'], config['activation']
 
    ### google the getattr function - eg: getattr(activation, 'relu')(junk) is same as activation.relu(junk)
    X_train, Y_train = data['train']['X'], data['train']['Y']
    # print(num_hidden_layers, hidden_layers_size)

    for i in range(num_epochs):
      dw = []
      db = []
      check = 0

      for k in tqdm(range(len(X_train))) :
        x = X_train[k]
        y = Y_train[k]
        temp = network.self_grad_wandb(x,y,activation_func)

        if check==0 :
          dw = temp['dw']
          db = temp['db']
          check=1

        else :
          for j in range(len(dw)) :
            dw[j] =dw[j] + temp['dw'][j]
            db[j] =db[j] + temp['db'][j]

        if (k+1)%batch_size == 0 :
          for dd in dw :
            dd *=eta
          for dd in db :
            dd *=eta
          network.update_vals(dw, db)
          check=0

  @staticmethod
  def sgd(network, data, config):

    # num_hidden_layers, hidden_layers_size = config['num_hidden_layers'], config['hidden_layer_size']
    num_epochs, batch_size = config['num_epochs'], config['batch_size']
    eta, lambda_ = config['lr'], config['weight_decay']
    initializer, activation_func = config['weights_initializer'], config['activation']
 
    ### google the getattr function - eg: getattr(activation, 'relu')(junk) is same as activation.relu(junk)
    X_train, Y_train = data['train']['X'], data['train']['Y']
    # print(num_hidden_layers, hidden_layers_size)

    for i in range(num_epochs):
      dw = []
      db = []
      for k in tqdm(range(0, len(X_train), batch_size)) :
        X = X_train[k: k+batch_size]
        Y = Y_train[k: k+batch_size]
        temp = network.self_grad_wandb(X, Y, activation_func)
        for j in range(len(temp['db'])) :
          temp['db'][j] = np.sum(temp['db'][j],axis=1)
          
        dw = temp['dw']
        db = temp['db']
        for dd in dw :
          dd *= eta
        for dd in db :
          dd*=eta

        network.update_vals(dw, db)
      

  @staticmethod
  def momentum(X,Y,max_epochs,eta,gamma,batch_size) :
    test = get_nn_shape()
    network = neural_network(test,test)
    dw = []
    db = []
    pred1 = []
    check = 0
    for i in range(max_epochs) :
      for k in tqdm(range(len(X))) :
        x = X[k]
        y = Y[k]
        temp = network.self_grad_wandb(x,y)
        if check==0 :
          dw = temp['dw']
          db = temp['db']
          for dd in db :
            dd*= eta
          for dd in dw :
            dd*=eta
          check = 1
        else :
          for j in range(len(dw)) :
            dw[j] += eta*temp['dw'][j]
            db[j] += eta*temp['db'][j]
        if (k+1) % batch_size == 0 or k == len(X)-1:
          network.update_vals(dw,db)
          for dd in db :
            dd *=gamma
          for dd in dw :
            dd*=gamma

      pred = 0
      for x,y in zip(X,Y) :
        pred = pred + (network.predict(x)!=y)

      print('error',pred)
    
    return pred1
  
  @staticmethod
  def NAG(X,Y,max_epochs,eta,gamma,batch_size) :
    test = get_nn_shape()
    network = neural_network(test,test)
    dw = []
    db = []
    check = 0
    for i in range(max_epochs) :
      for k in tqdm(range(len(X))) :
        x = X[k]
        y = Y[k]
        temp = network.self_grad_wandb(x,y)
        if check==0 :
          dw = temp['dw']
          db = temp['db']
          for dd in db :
            dd*= eta
          for dd in dw :
            dd*=eta
          check = 1
        else :
          for j in range(len(dw)) :
            dw[j] += eta*temp['dw'][j]
            db[j] += eta*temp['db'][j]

        if (k+1) % batch_size == 0 or k == len(X)-1:
          network.update_vals(dw,db)
          for dd in db :
            dd*=gamma
          for dd in dw :
            dd*=gamma
          network.update_vals(dw,db)

      pred = 0
      for x,y in zip(X,Y) :
        pred += (network.predict(x)!=y)

      print('error',pred)

  @staticmethod
  def RMSprop(X,Y,max_epochs,eta,beta,batch_size,epsilon) :
    test = get_nn_shape()
    network = neural_network(test,test)
    v_dw = []
    v_db = []
    check1 = 0
    for i in range(max_epochs) :
      dw = []
      db = []
      check = 0
      for k in tqdm(range(len(X))) :
        x = X[k]
        y = Y[k]
        temp = network.self_grad_wandb(x,y)
        if check==0 :
          dw = temp['dw']
          db = temp['db']
          check = 1
        else :
          for j in range(len(dw)) :
            dw[j] += temp['dw'][j]
            db[j] += temp['db'][j]

        if (k+1) % batch_size == 0 or k == len(X)-1:
          if check1==0 :
            for j in range(len(dw)) :
              v_dw.append( (1-beta)*(dw[j]**2) )
              dw[j] *= eta/np.sqrt(v_dw[j]+epsilon)

            for j in range(len(db)) :
              v_db.append( (1-beta)*(db[j]**2) )
              db[j] *= eta/np.sqrt(v_db[j]+epsilon)

            check1 = 1

          else :
            for j in range(len(dw)) :
              v_dw[j] *= beta
              v_dw[j] += (1-beta)*(dw[j]**2) 
              dw[j] *= eta/np.sqrt(v_dw[j]+epsilon)

            for j in range(len(db)) :
              v_db[j] *= beta
              v_db[j] += (1-beta)*(db[j]**2) 
              db[j] *= eta/np.sqrt(v_db[j]+epsilon)

          network.update_vals(dw,db)
          check = 1

      pred = 0
      for x,y in zip(X,Y) :
        pred += (network.predict(x)!=y)

      print('error',pred)


  @staticmethod
  def adam(X,Y,max_epochs,eta,beta1,beta2,batch_size) :
    test = get_nn_shape()
    network = neural_network(test,test)
    v_dw = []
    v_db = []
    m_dw = []
    m_db = []
    check1 = 0
    num_updates = 0
    for i in range(max_epochs) :
      dw = []
      db = []
      check = 0
      for k in tqdm(range(len(X))) :
        x = X[k]
        y = Y[k]
        temp = network.self_grad_wandb(x,y)
        if check==0 :
          dw = temp['dw']
          db = temp['db']
          check = 1
        else :
          for j in range(len(dw)) :
            dw[j] += temp['dw'][j]
            db[j] += temp['db'][j]

        if (k+1) % batch_size == 0 or k == len(X)-1:
          if check1==0 :
            for j in range(len(dw)) :
              v_dw.append( (1-beta1)*(dw[j]**2) )
              dw[j] *= eta/np.sqrt(v_dw[j]+epsilon)

            for j in range(len(db)) :
              v_db.append( (1-beta1)*(db[j]**2) )
              db[j] *= eta/np.sqrt(v_db[j]+epsilon)

            check1 = 1

          else :
            for j in range(len(dw)) :
              v_dw[j] *= beta1
              v_dw[j] += (1-beta1)*(dw[j]**2) 
              dw[j] *= eta/np.sqrt(v_dw[j]+epsilon)

            for j in range(len(db)) :
              v_db[j] *= beta1
              v_db[j] += (1-beta1)*(db[j]**2) 
              db[j] *= eta/np.sqrt(v_db[j]+epsilon)

          network.update_vals(dw,db)
          check = 1

      pred = 0
      for x,y in zip(X,Y) :
        pred += (network.predict(x)!=y)

      print('error',pred)




In [ ]:
# X = np.array([[1,1,2],[-1,2,3],[10,-67,43],[-5,45,-67]])
# Y = np.array([1,0,1,0])
# temp = solver.sgd(X,Y,100,1e-3)
X=data['train']['X']
# print(len(X))
# print(X[0])
Y = data['train']['y']
temp = solver.RMSprop(X,Y,100,0.001,0.9,32,1e-2)
print(temp)

In [85]:
nn = set_nn_shape()
network = neural_network(nn, 'random')
#data1 = {'train' : {'X': np.array([[1,1,1],[1,1,1],[1,1,1],[1,1,1],[1,1,1]]), 'Y' : np.array([0,1,3,2,1]) },'val' : {'X': np.array([[1,1,1],[1,1,1],[1,1,1],[1,1,1],[1,1,1]]), 'Y' : np.array([0,1,3,2,1]) }}
# need to change this bit later to accomodate other optimization functions
config1 = {'num_epochs' : 5,'lr' : 1e-4,'optimizer': 'sgd', 'batch_size' : 3 , 'weights_initializer' : 'random' , 'weight_decay' : 0.001, 'activation' : 'sigmoid' }
optimizer.sgd(network, data,config1 )

# generating reports for the run
report = run_callback(network, data, config1) 

print(report)


Number Of Hidden Layers:
3

Size Of Each Hidden Layer:
32

The Neural Network Has 5 Layers In Total!


TypeError: ignored

In [83]:
report = run_callback(network, data, config1) 

print(report)

{'loss': {'train': array([0.        , 0.        , 6.97576531]), 'val': array([0., 0., 0.])}, 'accuracy': {'train': 6.975765306122449, 'val': 0.0}}


In [ ]:
a = np.array([1, 0, 3])
b = np.zeros((a.size, a.max()+1))
b[np.arange(a.size),a] = 1
b.T

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 1.]])

In [82]:
def run_callback(network,data,config) :
    
    activation_func = config['activation']
    
    X_train = data['train']['X']
    Y_train = data['train']['Y']

    X_val = data['val']['X']
    Y_val = data['val']['Y']

    X_test = data['test']['X']
    Y_test = data['test']['Y']

    train_loss = 0
    train_count = 0
    train_sq_error = 0
    X_train = X_train.T.reshape((784,-1))
    temp = network.predict(X_train,activation_func)
    train_count = np.sum(np.argmax(temp['y_hat'],axis=0).reshape(Y_train.shape)==Y_train)
    #train_loss -= np.log(temp['y_hat'][y]) 
    #temp['y_hat'][y] = 1 - temp['y_hat'][y]
    #train_sq_error += np.sum(np.dot(temp['y_hat'],temp['y_hat']))

    val_loss = 0
    val_count = 0
    val_sq_error = 0
    # for x,y in zip(X_val,Y_val) :
    #     temp = network.predict(x,activation_func)
    #     if temp['y'] == y :
    #         val_count += 1
    #     val_loss -= np.log(temp['y_hat'][y]) 
    #     temp['y_hat'][y] = 1 - temp['y_hat'][y]
    #     val_sq_error += np.sum(np.dot(temp['y_hat'],temp['y_hat'])) 
    '''
    test_loss = 0
    test_count = 0
    test_sq_error = 0
    for x,y in zip(X_test,Y_test) :
        temp = network.predict(x,activation_func)
        if temp['y'] == y :
            test_count += 1
        test_loss -= np.log(temp['y_hat'][y]) 
        temp['y_hat'][y] = 1 - temp['y_hat'][y]
        test_sq_error += np.sum(np.dot(temp['y_hat'],temp['y_hat'])) 
    '''
    return  {
        'loss': {
            'train' : np.array([train_sq_error,train_loss,train_count])/len(X_train),
            'val' : np.array([val_sq_error,val_loss,val_count])/len(X_val)
        },
        'accuracy': {
            'train': train_count / len(X_train),
            'val': val_count / len(X_val)
        }
        #'test' : np.array([test_sq_error,test_loss,test_count])/len(X_test)
    }

    


In [16]:
np.argmax

0
3
6
9
12
15
18
21
24
27
30
33
36
39
42
45
48
